In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role
import pandas as pd

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = 'lawsnic-aiml-east2'
prefix = 'kaggle/customerChurn'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

training_dataset_s3_path = f"s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T17-24-14/part-00000-3986a971-e8a5-4844-bfc8-e72e78f341b4-c000.csv"

s3_output_location = 's3://{}/{}/autogluon/output'.format(bucket,prefix)

In [3]:
from sagemaker import image_uris, model_uris, script_uris

# Currently, not all the object detection models in jumpstart support finetuning. Thus, we manually select a model
# which supports finetuning.
train_model_id, train_model_version, train_scope = (
    "autogluon-classification-ensemble",
    "*",
    "training",
)
training_instance_type = "ml.g4dn.2xlarge"  # set a different GPU type to avoid instance insufficiency for p3 instance that is used by TabTransformer

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

In [4]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

hyperparameters["eval_metric"] = "roc_auc"
print(hyperparameters)

{'eval_metric': 'roc_auc', 'presets': 'medium_quality', 'auto_stack': 'False', 'num_bag_folds': '0', 'num_bag_sets': '1', 'num_stack_levels': '0', 'refit_full': 'False', 'set_best_to_refit_full': 'False', 'save_space': 'False', 'verbosity': '2'}


In [9]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base

training_job_name = name_from_base("jumpstart-example-churn-ag-g")

# Create SageMaker Estimator instance
tabular_estimator_ag = Estimator(
    role=role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
)


# Launch a SageMaker Training job by passing s3 path of the training data
ag = tabular_estimator_ag.fit(
    {"training": training_dataset_s3_path}, logs=True, job_name=training_job_name
)

2023-01-08 20:31:06 Starting - Starting the training job...
2023-01-08 20:31:29 Starting - Preparing the instances for trainingProfilerReport-1673209866: InProgress
.........
2023-01-08 20:32:50 Downloading - Downloading input data...
2023-01-08 20:33:30 Training - Downloading the training image.........
2023-01-08 20:34:50 Training - Training image download completed. Training in progress...2023-01-08 20:35:13,827 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2023-01-08 20:35:13,852 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"auto_stack":"False","eval_metric":"roc_auc","num_bag_folds":"0","num_bag_sets":"1","num_stack_levels":"0","presets":"medium_quality","refit_full":"False","save_space":"False","set_best_to_refit_full":"False","verbosity":"2"}', 'SM_USER_ENTRY_POINT': 'transfer_learning.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CO

In [12]:
#ag.fit_summary(show_plot=True)
ag